In [1]:
import soundfile as sf
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch

In [2]:
class CustomTTSHandler:
    def __init__(self, tts_model_dir, vocoder_model_dir):
        self.processor = SpeechT5Processor.from_pretrained(tts_model_dir)
        self.model = SpeechT5ForTextToSpeech.from_pretrained(tts_model_dir)
        self.vocoder = SpeechT5HifiGan.from_pretrained(vocoder_model_dir)
        
        # Load speaker embeddings
        embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
        self.speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    def predict(self, text):
        inputs = self.processor(text=text, return_tensors="pt")
        with torch.no_grad():
            speech = self.model.generate_speech(inputs["input_ids"], self.speaker_embeddings, vocoder=self.vocoder)
        return speech.numpy(), 16000  # numpy array와 sample rate를 반환합니다.


In [5]:
# 모델 디렉토리 지정
tts_model_dir = "/Users/lhe339/Documents/GitHub/TTS/SpeechT5/merged_model_directory/tts_model"  # Update this path
vocoder_model_dir = "/Users/lhe339/Documents/GitHub/TTS/SpeechT5/merged_model_directory/vocoder_model"  # Update this path

In [6]:
# CustomTTSHandler 인스턴스 생성
tts_handler = CustomTTSHandler(tts_model_dir, vocoder_model_dir)

In [7]:
# 텍스트를 음성으로 변환
text = "hi, my name is honey."
speech_output, samplerate = tts_handler.predict(text)

# 결과를 WAV 파일로 저장
sf.write('ST5.wav', speech_output, samplerate)